In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
from dateutil import parser
from datetime import datetime
%matplotlib inline

In [32]:
economic_df = pd.read_csv('C:\Alex_Code\quant_competetion-final\data\economic_indicator.csv', 
                          encoding='utf-8-sig', index_col='DateTime')
economic_df.index=economic_df.index.map(lambda x:str(x)[:-3])
economic_df.index = pd.DatetimeIndex(economic_df.index)
economic_df = economic_df[['CPI当月同比', '工业增加值当月同比']]

In [ ]:
print